In [ ]:
import pandas as pd
import numpy as np
from numpy import inf

In [ ]:
#Import a csv file with data about day, buyer_id, item_det_info, pay_timestamp and etc.
alidf = pd.read_csv('order_data_toy_1.csv', names = ['day', 'order_id','item_det_info','pay_timestamp','buyer_id','promise_speed','if_cainiao','merchant_id','Logistics_review_score'])

In [ ]:
#alidf

In [ ]:
#Convert column "pay_timestamp" to a datetime column and identify today's date
alidf['pay_timestamp'] = pd.to_datetime(alidf['pay_timestamp'])
alidf['today_timestamp']= pd.to_datetime('today')

In [ ]:
#Check the data types for all columns
alidf.dtypes

In [ ]:
#Calculate "recency"
alidf['recency'] = alidf['today_timestamp'] - alidf['pay_timestamp']

In [ ]:
#alidf['recency']
#alidf.iloc[56]

In [ ]:
#Calculate revenue for each purchase
s =[]
for row in range(len(alidf)):
    items = alidf['item_det_info'][row].split(';')
    #print(items)
    for i in range(len(items)):
        revenue = 0 
        si = items[i]
        item_info = si.split(':')
        if len(item_info) == 3:
            revenue += float (item_info[2])
        else:
            #print('incomplete data:', si)
            pass
    s.append(revenue)
print(s)

In [ ]:
#Create a Series for revenue data
se = pd.Series(s)
alidf['revenue'] = se.values

In [ ]:
#Check revenue data in alidf
alidf['revenue']

In [ ]:
#alidf

In [ ]:
#Calculate revenue on each buyer_id
alidf2 = alidf.groupby(['buyer_id'])['revenue'].sum()
alidf2

In [ ]:
#Calculate purchase frequency of each buyer_id
alidf3 = alidf.groupby(['buyer_id'])['pay_timestamp'].count()
alidf3

In [ ]:
#Calculate recency of each buyer_id
alidf4 = alidf.groupby(['buyer_id'])['recency'].min()
alidf4

In [ ]:
#Create a new DataFrame by concating DataFrames about revenue, purchase and recency, and rename columns
rfmTable = pd.concat([alidf2, alidf3, alidf4], axis=1).reset_index()
rfmTable.columns = ['buyer_id', 'monetary', 'frequency','recency']

In [ ]:
#Convert "recency" data to days
rfmTable.recency = rfmTable.recency.dt.days

In [ ]:
#Split metrics into segments by using quartiles
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [ ]:
#Create a segmented RFM table
segmented_rfm = rfmTable

In [ ]:
#Calculate Recency scores and FM scores
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
#Add segment numbers to the newly created segmented RFM table
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary'].apply(FMScore, args=('monetary',quantiles,))
segmented_rfm

In [ ]:
#Add a new column to combine RFM score: 141 is the highest score as we saw from the segmented_rfm table
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)

In [ ]:
#Find out the top 10 of our best customers
segmented_rfm[segmented_rfm['RFMScore']=='141'].sort_values('monetary', ascending=False).head(10)